In [1]:
import torch


In [3]:



from torch import nn, optim
from torch.utils.data import DataLoader 
from torchvision import datasets, transforms
import matplotlib.pyplot as plt

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_dataset = datasets.MNIST(
    root = 'data',
    train = True,
    download = True,
    transform = transform
)
train_loader = DataLoader(train_dataset, batch_size = 42, shuffle = True)

100%|██████████| 28.9k/28.9k [00:00<00:00, 91.6kB/s]
100%|██████████| 1.65M/1.65M [00:01<00:00, 1.11MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 505kB/s]


In [4]:
class Generator(nn.Module):
    def __init__(self, z_dim):
        super(Generator,self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(z_dim, 256),
            nn.ReLU(True),
            nn.Linear(256, 512),
            nn.ReLU(True),
            nn.Linear(512, 1024),
            nn.ReLU(True),
            nn.Linear(1024, 784),
            nn.Tanh()
        )
    def forward(self, x):
        return self.fc(x).view(-1, 1, 28, 28)

In [5]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator,self).__init__()
        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(28*28, 1024),
            nn.LeakyReLU(0.2),
            nn.Linear(1024, 512),
            nn.LeakyReLU(0.2),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2),
            nn.Linear(256, 1),
            nn.Sigmoid()
        )
    def forward(self, x):
        return self.fc(x)

In [7]:
criterion = nn.BCELoss()
z_dim = 100
lr = 0.0002
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
G = Generator(z_dim).to(device)
D = Discriminator().to(device)

optimizer_G = optim.Adam(G.parameters(), lr = lr, betas = (0.5, 0.999))
optimizer_D = optim.Adam(D.parameters(), lr = lr, betas = (0.5, 0.999))